In [1]:
import pandas as pd
import numpy as np
import astropy.utils.data
from astropy.coordinates import SkyCoord,concatenate
import astropy.units as u
import datetime as _datetime
from astropy.io import fits as _fits
from astropy.coordinates import FK5,ICRS
from astropy.io import ascii
from astropy.coordinates import solar_system_ephemeris
from astropy.time import Time

# Join results tables together

In [2]:
df_200219=pd.read_csv('200219_results.csv')
df_191213=pd.read_csv('191213g_results.csv')
final_results=pd.concat([df_200219,df_191213], axis=0)
final_results=final_results.reset_index(drop=True)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2,3,4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2,3,4,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
final_results_ps1=final_results[final_results['source']=='PS1']

final_results_ps1[['our_id','RA','Dec']].to_csv('ps1_sources_for_gaia.csv')
final_results[['our_id','source','RA','Dec']].to_csv('all_sources_for_gaia.csv')

# Load in xmatch

In [4]:
xmatch_ps1_only=pd.read_csv('gaia_ps1_xmatch_result.csv')
xmatch_all=pd.read_csv('all_xmatch.csv')

# Identify sources not crossmatched

In [5]:
common = final_results.merge(xmatch_all,on=['our_id','our_id'])

not_xmatched=final_results[(~final_results.our_id.isin(common.our_id))&(~final_results.our_id.isin(common.our_id))]


# Cut on astrometric error and proper motion

In [6]:
xmatch_all['uwe']=np.sqrt(xmatch_all['astrometric_chi2_al']/(xmatch_all['astrometric_n_good_obs_al']-5))
print(min(xmatch_all['uwe']), max(xmatch_all['uwe']), np.nanmin(xmatch_all['pmra']), np.nanmax(xmatch_all['pmdec']))
#and cut on pmra pmdec


0.7415034052517898 77.42827387917109 -74.76479400626064 70.13181403174896


In [7]:
print(np.mean(xmatch_all['uwe']),np.mean(xmatch_all['pmra']))

2.1178473513618283 -2.2904165896792326


In [8]:
xmatch_ps1_only=xmatch_all[xmatch_all['source']=='PS1']
xmatch_GLADE_only=xmatch_all[xmatch_all['source']=='GLADE']

xmatch_cut=xmatch_ps1_only[(xmatch_ps1_only['uwe']>1.5) & (np.abs(xmatch_ps1_only['pmra']<3) | (xmatch_ps1_only['pmra']==np.nan)) & (np.abs(xmatch_ps1_only['pmdec']<3) | (xmatch_ps1_only['pmdec']==np.nan))]


# Recombine gaia-limited df with ps1/GLADE columns

In [9]:
xmatch_glade_ps1_cut=pd.concat([xmatch_cut,xmatch_GLADE_only], axis=0)
print(len(xmatch_glade_ps1_cut))

6572


# If there are duplicate PS1 crossmatches, only take closest one

In [10]:
def sep(ra,dec,ra1,dec1):
    #c1 = SkyCoord(ra=ra, '-69d45m22s', frame='icrs')
    c1=SkyCoord(ra*u.deg, dec*u.deg)
    c2=SkyCoord(ra1*u.deg, dec1*u.deg)
    sep = c1.separation(c2)
    return sep.arcsecond

xmatch_glade_ps1_cut['separation']=np.abs(sep(xmatch_glade_ps1_cut['ra'].values,xmatch_glade_ps1_cut['dec'].values,xmatch_glade_ps1_cut['ra.1'].values,xmatch_glade_ps1_cut['dec.1'].values))
xmatch_glade_ps1_cut=xmatch_glade_ps1_cut.sort_values('separation').drop_duplicates(subset='our_id', keep='first')

xmatch_glade_ps1_cut['RA']=xmatch_glade_ps1_cut['ra']
xmatch_glade_ps1_cut['Dec']=xmatch_glade_ps1_cut['dec']
#

# Merge xmatch with non-xmatch

In [11]:
total_cut_df = pd.concat([not_xmatched,xmatch_glade_ps1_cut])

In [12]:
print(list(total_cut_df.columns))

['Unnamed: 0', 'PGC', 'GWGC', 'HyperLEDA', '2MASS', 'SDSS', 'flag1', 'RA', 'Dec', 'dist', 'dist_err', 'z', 'Bmag', 'e_Bmag', 'AbsBMag', 'Jmag', 'e_Jmag', 'Hmag', 'e_Hmag', 'Kmag', 'eKmag', 'flag2', 'flag3', 'source', 'objName', 'objNameHMS', 'objAltName1', 'objAltName2', 'objAltName3', 'objPopularName', 'objID', 'uniquePspsOBid', 'ippObjID', 'surveyID', 'htmID', 'zoneID', 'tessID', 'projectionID', 'skyCellID', 'randomID', 'batchID', 'dvoRegionID', 'processingVersion', 'objInfoFlag', 'qualityFlag', 'raStack', 'decStack', 'raStackErr', 'decStackErr', 'raMeanErr', 'decMeanErr', 'epochMean', 'posMeanChisq', 'cx', 'cy', 'cz', 'lambda', 'beta', 'l', 'b', 'nStackObjectRows', 'nStackDetections', 'nDetections', 'ng', 'nr', 'ni', 'nz', 'ny', 'Column1', 'Column2', 'gQfPerfect', 'gMeanPSFMag', 'gMeanPSFMagErr', 'gMeanPSFMagStd', 'gMeanPSFMagNpt', 'gMeanPSFMagMin', 'gMeanPSFMagMax', 'gMeanKronMag', 'gMeanKronMagErr', 'gMeanKronMagStd', 'gMeanKronMagNpt', 'gMeanApMag', 'gMeanApMagErr', 'gMeanApMagSt

# Template FITS

In [13]:
template_hdulist = _fits.open('WS2021A1-003_CatalogueTemplate.fits')
template_primary_hdu = template_hdulist[0]
template_hdu = template_hdulist[1]
template_column_names = [col.name for col in template_hdu.columns]
print(template_column_names)


['CNAME', 'TARGSRVY', 'TARGPROG', 'TARGCAT', 'TARGID', 'TARGNAME', 'TARGPRIO', 'TARGUSE', 'TARGCLASS', 'PROGTEMP', 'OBSTEMP', 'GAIA_ID', 'GAIA_DR', 'GAIA_RA', 'GAIA_DEC', 'GAIA_EPOCH', 'GAIA_PMRA', 'GAIA_PMRA_ERR', 'GAIA_PMDEC', 'GAIA_PMDEC_ERR', 'GAIA_PARAL', 'GAIA_PARAL_ERR', 'HEALPIX', 'IFU_SPAXEL', 'IFU_PA', 'IFU_DITHER', 'MAG_G', 'MAG_G_ERR', 'MAG_R', 'MAG_R_ERR', 'MAG_I', 'MAG_I_ERR', 'GAIA_MAG_G', 'GAIA_MAG_G_ERR', 'GAIA_MAG_BP', 'GAIA_MAG_BP_ERR', 'GAIA_MAG_RP', 'GAIA_MAG_RP_ERR', 'APS_WL_MIN', 'APS_WL_MAX', 'APS_Z', 'APS_SIGMA', 'APS_TEMPL_LIB', 'APS_TEMPL_LIB_NORM', 'APS_PPXF_WL_MIN', 'APS_PPXF_WL_MAX', 'APS_PPXF_MOM', 'APS_PPXF_DEG_ADD', 'APS_PPXF_DEG_MULT', 'APS_PPXF_NUM_MC', 'APS_GAND_MODE', 'APS_GAND_ERR', 'APS_GAND_RED1', 'APS_GAND_RED2', 'APS_GAND_EBV', 'APS_LS_MODE', 'APS_LS_RES', 'APS_LS_NUM_MC', 'APS_SSP_NUM_WLKR', 'APS_SSP_NUM_CHAIN', 'APS_IFU_MASK', 'APS_IFU_TSSL_TYPE', 'APS_IFU_TSSL_TARG_SNR', 'APS_IFU_TSSL_MIN_SNR', 'APS_IFU_TSSL_COVAR', 'APS_IFU_SRC_ID', 'APS_IF

## Calculate gaia mag errors

In [14]:
# according to dr2 https://www.cosmos.esa.int/documents/29201/1773953/Gaia+DR2+primer+version+1.3.pdf/a4459741-6732-7a98-1406-a1bea243df79?t=1581668739161
total_cut_df['phot_g_mean_mag_err']=1.086/total_cut_df['phot_g_mean_flux_over_error']
total_cut_df['phot_bp_mean_mag_err']=1.086/total_cut_df['phot_bp_mean_flux_over_error']
total_cut_df['phot_rp_mean_mag_err']=1.086/total_cut_df['phot_rp_mean_flux_over_error']

## Take relevant columns from our catalogue

In [18]:
total_cut_df = total_cut_df.reset_index(drop=True)
FITS_table_from_cat=total_cut_df[['source','our_id','source_id','ra.1', 'dec.1', 'ref_epoch', 'pmra', 'pmra_error',
                                  'pmdec', 'pmdec_error','parallax', 'parallax_error','phot_g_mean_mag',
                                  'phot_g_mean_mag_err','phot_bp_mean_mag','phot_bp_mean_mag_err',
                                  'phot_rp_mean_mag','phot_rp_mean_mag_err', 'gMeanKronMag', 'gMeanKronMagErr',
                                  'rMeanKronMag', 'rMeanKronMagErr','iMeanKronMag', 'iMeanKronMagErr',
                                  'objID',
                                  'gMeanKronMag', 'gMeanKronMagErr','iMeanKronMag', 'iMeanKronMagErr',
                                  'rMeanKronMag', 'rMeanKronMagErr','yMeanKronMag', 'yMeanKronMagErr','zMeanKronMag', 'zMeanKronMagErr']]


FITS_table_from_cat.columns = ['COORDCAT','TARGID','GAIA_ID','GAIA_RA', 'GAIA_DEC', 'GAIA_EPOCH', 'GAIA_PMRA', 'GAIA_PMRA_ERR', 
                               'GAIA_PMDEC', 'GAIA_PMDEC_ERR', 'GAIA_PARAL', 'GAIA_PARAL_ERR','GAIA_MAG_G', 
                               'GAIA_MAG_G_ERR', 'GAIA_MAG_BP', 'GAIA_MAG_BP_ERR', 
                               'GAIA_MAG_RP', 'GAIA_MAG_RP_ERR', 'MAG_G', 'MAG_G_ERR', 
                               'MAG_R', 'MAG_R_ERR', 'MAG_I', 'MAG_I_ERR',
                               'PS_ID', 
                               'PS_MAG_G', 'PS_MAG_G_ERR', 'PS_MAG_I', 'PS_MAG_I_ERR', 
                               'PS_MAG_R', 'PS_MAG_R_ERR', 'PS_MAG_Y', 'PS_MAG_Y_ERR', 'PS_MAG_Z', 'PS_MAG_Z_ERR' ]

FITS_table_from_cat.reset_index()
#survey identifier
FITS_table_from_cat['TARGSRVY']='WS2021A1-003'

#optional personal name for programme/catalogue
FITS_table_from_cat['TARGPROG']='GW-galaxy'
FITS_table_from_cat['TARGCAT']='GW-galaxy'

#T for target, GALAXY
FITS_table_from_cat['TARGUSE']='T'
FITS_table_from_cat['TARGCLASS']='GALAXY'

# PROGTEMP for MOS LR 3x1020s 60min OB
FITS_table_from_cat['PROGTEMP']=11331

#OBSTEMP for max airmass=2.4, moon dist=90 deg, 
#max sky brightness=20.5, max seeing = 1.3'', transparency=0.8
FITS_table_from_cat['OBSTEMP']='GAFAD'

#TARGNAME is the same as unique id
FITS_table_from_cat['TARGNAME']=FITS_table_from_cat['TARGID']

#TARGPRIO = 10.0 for the moment
FITS_table_from_cat['TARGPRIO']=10.0

#Gaia DR
FITS_table_from_cat['GAIA_DR']='PS1'

#Optional coordinates provenance columns

#PS1 and GLADE V2
def version (df):
    if df['source'] == 'GLADE' :
        return 'V2'
    else:
        return 'PS1'
FITS_table_from_cat['COORDCAT_DR']=total_cut_df.apply (lambda total_cut_df: version(total_cut_df), axis=1)
#FITS_table_from_cat['COORDCAT_DR']='2'

#don't know epoch of GLADE, so just give epoch to PS1
FITS_table_from_cat['COORDCAT_EPOCH']=total_cut_df['epochMean']

#ID from GLADE or PS1
def cat_id(df):
    if df['source'] == 'GLADE' :
        return df['SDSS']
    else:
        return df['objID']
FITS_table_from_cat['COORDCAT_ID']=total_cut_df.apply (lambda total_cut_df: cat_id(total_cut_df), axis=1)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [19]:
# Take PS1 RA and Dec
FITS_table_from_cat['PS_RA']=total_cut_df['ra']
FITS_table_from_cat['PS_DEC']=total_cut_df['dec']
FITS_table_from_cat['PS_DR']='2'


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

## If we don't have Gaia RA/Dec, transform PS1/GLADE to J2015.5

In [20]:
# ingest the coordinates
c = SkyCoord(ra=total_cut_df['RA'], dec=total_cut_df['Dec'], unit='deg', frame='fk5')

# precess to J2015.5, i.e., Gaia reference system
with solar_system_ephemeris.set('jpl'): 
    c_Gaia = c.transform_to(ICRS(obstime=Time('J2015.5')))

FITS_table_from_cat = FITS_table_from_cat.reset_index(drop=True)

TypeError: Coordinate frame got unexpected keywords: ['obstime']